In [33]:
import numpy as np
import os
import pickle
import wfdb.processing

# array of id of record, file name of each ECG
record_ids = []
# array where each element is signal for ECG file
records = []

for file in os.listdir('./ECGDataDenoised'):
    a = file.split('.')
    if a[1] != 'csv':
        # skip non csv files
        continue
    # read signal csv file
    rec_orig = []
    with open('./ECGDataDenoised/' + file) as f:
        reader = csv.reader(f)
        for row in reader:
            rec_orig.append(row[0])
    f.close()
    rec_100_hz = []
    if len(rec_orig) == 5000:
        # resample to 100 Hz
        rec_100_hz = scipy.signal.resample(rec_orig, 1000)
    record_ids.append(a[0])
    records.append(rec_100_hz)

with open('record_ids_100.pkl', 'wb') as f:
    pickle.dump(record_ids,f)
    f.close()

with open('records_100.pkl', 'wb') as f:
    pickle.dump(records,f)
    f.close()
    
print('Done :)')



Done :)


In [34]:
import pickle
import csv

with open('record_ids_100.pkl', 'rb') as f:
    record_ids = pickle.load(f)
    f.close()

with open('records_100.pkl', 'rb') as f:
    records = pickle.load(f)
    f.close()

# record dictionary w record ids and corresponding record locations in array
record_dict = {}

for i in range(0,len(record_ids)):
    record_dict[record_ids[i]] = i

# atrial flutter ECGs
AF_files = []
# sinus irregularity ECGs
SA_files = []
# supraventricular tachycarida ECGs
SVT_files = []
# atrial tachycardia ECGs
AT_files = []

# read annotations from annotation csv file
with open('Annotations.csv', 'r', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        rec_id = row[0]
        rec_ann = row[1]
        rec_data = np.array([[float(s)] for s in records[record_dict[rec_id]]])
        if len(rec_data) != 1000 | (rec_ann == 'AVNRT') | (rec_ann == 'AVRT') | (rec_ann == 'SAAWR'):
            # get rid of samples we don't understand :)
            print('Skipped sample', rec_id, 'with annotation', rec_ann, 'and length', len(rec_data))
        # GET ATRIAL FLUTTER FILES
        elif rec_ann == 'AF':
            AF_files.append(rec_data)
        # GET SINUS IRREGULARITY FILES
        elif rec_ann == 'SA':
            SA_files.append(rec_data)
        # GET SUPRAVENTRICULAR TACHYCARDIA FILES
        elif rec_ann == 'SVT':
            SVT_files.append(rec_data)
        # GET ATRIAL TACHYCARDIA FILES
        elif rec_ann == 'AT':
            AT_files.append(rec_data)   
    csvfile.close()
        
print('We have', len(AF_files), 'atrial flutter samples')
print('We have', len(SA_files), 'sinus irregularity samples')
print('We have', len(SVT_files), 'supraventricular tachycardia samples')
print('We have', len(AT_files), 'atrial tachycardia samples')

with open('AF_files_100.pkl', 'wb') as f:
    pickle.dump(AF_files, f)
    f.close()

with open('SA_files_100.pkl', 'wb') as f:
    pickle.dump(SA_files, f)
    f.close()

with open('SVT_files_100.pkl', 'wb') as f:
    pickle.dump(SVT_files, f)
    f.close()

with open('AT_files_100.pkl', 'wb') as f:
    pickle.dump(AT_files, f)
    f.close()


Skipped sample MUSE_20180113_124215_52000 with annotation SB and length 0
Skipped sample MUSE_20180112_072334_92000 with annotation AVNRT and length 1000
Skipped sample MUSE_20180114_065933_39000 with annotation SAAWR and length 1000
Skipped sample MUSE_20180114_075630_41000 with annotation SAAWR and length 1000
Skipped sample MUSE_20180114_125927_34000 with annotation SAAWR and length 1000
Skipped sample MUSE_20180114_131536_74000 with annotation AVNRT and length 1000
Skipped sample MUSE_20180115_130314_40000 with annotation SAAWR and length 1000
Skipped sample MUSE_20180116_124311_28000 with annotation SAAWR and length 1000
Skipped sample MUSE_20180116_125256_19000 with annotation AVRT and length 1000
Skipped sample MUSE_20180116_130743_52000 with annotation AVNRT and length 1000
Skipped sample MUSE_20180116_133202_99000 with annotation SAAWR and length 1000
Skipped sample MUSE_20180116_133929_81000 with annotation AVNRT and length 1000
Skipped sample MUSE_20180116_133937_86000 with 

In [23]:
rec_41000_data = [[float(a0)] for a0 in records[record_dict['MUSE_20180712_153037_41000']]]
len(rec_41000_data)
with open('rec_41000_250.pkl', 'wb') as f:
    pickle.dump(rec_41000_data, f)
    f.close()

In [ ]:
import numpy as np
import os
import pickle
import csv

# pickle to csv files (all record_ids and all records)
with open

In [21]:
import csv

# ECG samples condense to a single array
all_ECG_files = []

# ECG labels (annotations) in one array
all_ECG_anns = []

# read annotations from annotation csv file
with open('Annotations.csv', 'r', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        rec_id = row[0]
        rec_ann = row[1]
        if (rec_ann == 'AVNRT') | (rec_ann == 'AVRT') | (rec_ann == 'SAAWR'):
            # get rid of samples we don't understand :)
            print('Skipped sample', rec_id, 'with annotation', rec_ann)
        else:
            # read ECG data csv file
            with open('ECGDataDenoised/'+ rec_id + '.csv') as ecg_csvfile:
                ecg_reader = csv.reader(ecg_csvfile)
                # extract lead 1 from ECG file
                rec_data = [float(s[0]) for s in ecg_reader]
                ecg_csvfile.close()
                if len(rec_data) == 5000:
                    # make single array w 1-lead ECGs
                    all_ECG_files.append(rec_data)
                    # make single array file containing all ECG annotations
                    all_ECG_anns.append(rec_ann)
                else:
                    print('Skipped sample', rec_id, 'with length', len(rec_data),'(expected 5000)')
    csvfile.close()

print('Got', len(all_ECG_files), 'ECG files')
print('Got', len(all_ECG_anns), 'ECG annotations')


Skipped sample MUSE_20180113_124215_52000 with length 1926 (expected 5000)


KeyboardInterrupt: 

In [56]:
import csv
import numpy as np

# atrial flutter ECGs
AF_files = []
AF_anns = []
# sinus irregularity ECGs
SA_files = []
SA_anns = []
# supraventricular tachycarida ECGs
SVT_files = []
SVT_anns = []
# atrial tachycardia ECGs
AT_files = []
AT_anns = []

# read annotations from annotation csv file
with open('Annotations.csv', 'r', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        rec_id = row[0]
        rec_ann = row[1]
        if (rec_ann == 'AVNRT') | (rec_ann == 'AVRT') | (rec_ann == 'SAAWR'):
            # get rid of samples we don't understand :)
            print('Skipped sample', rec_id, 'with annotation', rec_ann)
        # GET ATRIAL FLUTTER FILES/ANNOTATIONS
        elif rec_ann == 'AF':
            # read ECG data csv file
            with open('ECGDataDenoised/'+ rec_id + '.csv') as ecg_csvfile:
                ecg_reader = csv.reader(ecg_csvfile)
                # extract lead 1 from ECG file
                rec_data = np.array([[float(s[0])] for s in ecg_reader])
                ecg_csvfile.close()
                if len(rec_data) == 5000:
                    # make single array w 1-lead ECGs
                    AF_files.append(rec_data)
                    # make single array file containing all ECG annotations
                    AF_anns.append(rec_ann)
        # GET SINUS IRREGULARITY FILES/ANNOTATIONS
        elif rec_ann == 'SA':
            # read ECG data csv file
            with open('ECGDataDenoised/'+ rec_id + '.csv') as ecg_csvfile:
                ecg_reader = csv.reader(ecg_csvfile)
                # extract lead 1 from ECG file
                rec_data = np.array([[float(s[0])] for s in ecg_reader])
                ecg_csvfile.close()
                if len(rec_data) == 5000:
                    # make single array w 1-lead ECGs
                    SA_files.append(rec_data)
                    # make single array file containing all ECG annotations
                    SA_anns.append(rec_ann)
        # GET SUPRAVENTRICULAR TACHYCARDIA FILES/ANNOTATIONS
        elif rec_ann == 'SVT':
            # read ECG data csv file
            with open('ECGDataDenoised/'+ rec_id + '.csv') as ecg_csvfile:
                ecg_reader = csv.reader(ecg_csvfile)
                # extract lead 1 from ECG file
                rec_data = np.array([[float(s[0])] for s in ecg_reader])
                ecg_csvfile.close()
                if len(rec_data) == 5000:
                    # make single array w 1-lead ECGs
                    SVT_files.append(rec_data)
                    # make single array file containing all ECG annotations
                    SVT_anns.append(rec_ann)
        # GET ATRIAL TACHYCARDIA FILES/ANNOTATIONS
        elif rec_ann == 'AT':
            # read ECG data csv file
            with open('ECGDataDenoised/'+ rec_id + '.csv') as ecg_csvfile:
                ecg_reader = csv.reader(ecg_csvfile)
                # extract lead 1 from ECG file
                rec_data = np.array([[float(s[0])] for s in ecg_reader])
                ecg_csvfile.close()
                if len(rec_data) == 5000:
                    # make single array w 1-lead ECGs
                    AT_files.append(rec_data)
                    # make single array file containing all ECG annotations
                    AT_anns.append(rec_ann)        
                else:
                    print('Skipped sample', rec_id, 'with length', len(rec_data),'(expected 5000)')
    csvfile.close()
        
print('We have', len(AF_files), 'atrial flutter samples and', len(AF_anns), 'annotations')
print('We have', len(SA_files), 'sinus irregularity samples and', len(SA_anns), 'annotations')
print('We have', len(SVT_files), 'supraventricular tachycardia samples and', len(SVT_anns), 'annotations')
print('We have', len(AT_files), 'atrial tachycardia samples and', len(AT_anns), 'annotations')


Skipped sample MUSE_20180112_072334_92000 with annotation AVNRT
Skipped sample MUSE_20180114_065933_39000 with annotation SAAWR
Skipped sample MUSE_20180114_075630_41000 with annotation SAAWR
Skipped sample MUSE_20180114_125927_34000 with annotation SAAWR
Skipped sample MUSE_20180114_131536_74000 with annotation AVNRT
Skipped sample MUSE_20180115_130314_40000 with annotation SAAWR
Skipped sample MUSE_20180116_124311_28000 with annotation SAAWR
Skipped sample MUSE_20180116_125256_19000 with annotation AVRT
Skipped sample MUSE_20180116_130743_52000 with annotation AVNRT
Skipped sample MUSE_20180116_133202_99000 with annotation SAAWR
Skipped sample MUSE_20180116_133929_81000 with annotation AVNRT
Skipped sample MUSE_20180116_133937_86000 with annotation AVRT
Skipped sample MUSE_20180119_180042_46000 with annotation SAAWR
Skipped sample MUSE_20180712_150812_72000 with annotation AVRT
Skipped sample MUSE_20180712_150854_11000 with annotation AVRT
Skipped sample MUSE_20180712_150931_83000 wi

In [24]:
with open('ECGDataDenoised/'+ 'MUSE_20180712_153037_41000' + '.csv') as ecg_csvfile:
    ecg_reader = csv.reader(ecg_csvfile)
    # extract lead 1 from ECG file
    rec_41000_data = np.array([[float(s[0])] for s in ecg_reader])
    ecg_csvfile.close()

with open('rec_41000.pk', 'wb') as f:
    pickle.dump(rec_41000_data, f)
    f.close()

In [57]:
import pickle

with open('AF_files.pkl', 'wb') as f:
    pickle.dump(AF_files, f)
    f.close()

with open('AF_anns.pkl', 'wb') as f:
    pickle.dump(AF_anns, f)
    f.close()

with open('SA_files.pkl', 'wb') as f:
    pickle.dump(SA_files, f)
    f.close()

with open('SA_anns.pkl', 'wb') as f:
    pickle.dump(SA_anns, f)
    f.close()

with open('SVT_files.pkl', 'wb') as f:
    pickle.dump(SVT_files, f)
    f.close()

with open('SVT_anns.pkl', 'wb') as f:
    pickle.dump(SVT_anns, f)
    f.close()

with open('AT_files.pkl', 'wb') as f:
    pickle.dump(AT_files, f)
    f.close()

with open('AT_anns.pkl', 'wb') as f:
    pickle.dump(AT_anns, f)
    f.close()

In [ ]:
# one-hot encoding annotations

import numpy as np
import sklearn
encoder = sklearn.preprocessing.OneHotEncoder(sparse=False)

all_ECG_anns_arr = np.array(all_ECG_anns).reshape(-1, 1)
all_ECG_anns_onehot = encoder.fit_transform(all_ECG_anns_arr)
encoder.categories_

AttributeError: module 'sklearn' has no attribute 'preprocessing'

In [ ]:
# 5-fold CV to split train/test sets
import array as arr
from sklearn.model_selection import KFold
import numpy as np
import tensorflow as tf
from keras.utils import np_utils
from keras.metrics import Precision, Recall, TrueNegatives, TruePositives, FalseNegatives, FalsePositives
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv1D, MaxPooling1D, Reshape, GlobalAveragePooling1D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from datetime import datetime
import csv

now = datetime.now()
model_name_prefix = now.strftime("%y%m%d_%H%M") + '_'
print('Using', model_name_prefix, 'as model_name_prefix')

fold_number = 1
train_set_pieces_arr = []
kfold = KFold(n_splits=5, shuffle=True)
for train_set, test_set in kfold.split(all_ECG_files, all_ECG_anns_onehot):
    # train set
    #input - train set single lead ECGs to numpy array
    train_set_pieces_arr = np.array([all_ECG_files[i] for i in train_set ])
    train_set_pieces = train_set_pieces_arr.reshape(train_set_pieces_arr.shape[0], 5000, 1)
    print('train_set_pieces shape: ', train_set_pieces.shape)

    # output - train set target labels
    train_set_targets = np.array([all_ECG_anns_onehot[i] for i in train_set])
    print('train_set_targets shape: ', train_set_targets.shape)

    # test set
    # input - test set single lead ECGs to numpy array
    test_set_pieces_arr = np.array([all_ECG_files[i] for i in test_set])
    test_set_pieces = test_set_pieces_arr.reshape(test_set_pieces_arr.shape[0], test_set_pieces_arr.shape[1], 1)
    print('test_set_pieces shape: ', test_set_pieces.shape)

    # output - test set target labels
    test_set_targets = np.array([all_ECG_anns_onehot[i] for i in test_set])
    print('test_set_targets shape: ', test_set_targets.shape)


    # multi-class CNN model take 1
    # increase filters from 100 to 125, decrease kernel size from 36 to 18
    filters = 125
    kernel_size = 18
    cnn_1 = Sequential()
    cnn_1.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(5000, 1)))
    cnn_1.add(Conv1D(filters, kernel_size, activation='relu'))
    cnn_1.add(MaxPooling1D(3))
    cnn_1.add(Conv1D(filters, kernel_size, activation='relu'))
    cnn_1.add(Conv1D(filters, kernel_size, activation='relu'))
    cnn_1.add(GlobalAveragePooling1D())
    cnn_1.add(Dropout(0.5))
    cnn_1.add(Dense(activation='softmax', units=8))
    print(cnn_1.summary())

    # CNN model performance evaluation
    callbacks_list = [
    #    ModelCheckpoint(
    #        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
    #        monitor='val_loss', save_best_only=True),
        EarlyStopping(monitor='accuracy', patience=2)
    ]
    cnn_1.compile(loss='categorical_crossentropy',
                    optimizer='adam', metrics=['accuracy', Precision(), Recall(), TrueNegatives(), TruePositives(), FalseNegatives(), FalsePositives(), 'AUC'])

    history = cnn_1.fit(train_set_pieces,
                        train_set_targets,
                        batch_size=24,
                        epochs=20,
                        callbacks=callbacks_list,
                        validation_split=0,
                        verbose=2)
    print('Training metrics:', history.history)

    results = cnn_1.evaluate(
        test_set_pieces, test_set_targets, verbose=2, sample_weight=None, return_dict=False)
    print('Test metrics:', results)

    # save results
    SavedResults = os.path.join("SavedResults", model_name_prefix)
    if not os.path.exists(SavedResults):
        os.mkdir(SavedResults)

    # save model
    SavedModels = os.path.join("SavedModels")
    if not os.path.exists(SavedModels):
        os.mkdir(SavedModels)

    cnn_1.save('SavedModels/' + model_name_prefix + str(fold_number))

    # run test set to get individual metrics
    predicted_ann = cnn_1.predict(test_set_pieces, batch_size=5000)
    
    
    # write expected/actual classifications to csv file 
    with open('SavedResults/'+ model_name_prefix + '/results_' + str(fold_number) + '.csv', mode='w') as results_csvfile:
        writer = csv.writer(results_csvfile)
        for i in range(len(test_set)):
            predicted = encoder.categories_[0][np.argmax(predicted_ann[i])]
            writer.writerow([ test_set[i], all_ECG_anns[test_set[i]], predicted])
        results_csvfile.close()
    
    fold_number = fold_number + 1

Using 210823_1609_ as model_name_prefix
train_set_pieces shape:  (8491, 5000, 1)
train_set_targets shape:  (8491, 8)
test_set_pieces shape:  (2123, 5000, 1)
test_set_targets shape:  (2123, 8)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 4983, 125)         2375      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4966, 125)         281375    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1655, 125)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1638, 125)         281375    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1621, 125)         281375    
______________________________________________________________

In [ ]:
# acccess results for each individual class
import csv

# row[0] = index of sample
# row[1] = true label
# row[2] = predicted label

confusion = dict()
for l in encoder.categories_[0]:
    confusion[l] = dict()
    for m in encoder.categories_[0]:
        confusion[l][m] = 0

csvs = ['results_1.csv', 'results_2.csv', 'results_3.csv', 'results_4.csv', 'results_5.csv']
for c in csvs:
    csvfilename = 'SavedResults/'+ model_name_prefix + '/' + c
    print('reading from', csvfilename)
    with open(csvfilename) as results_csvfile:
        reader = csv.reader(results_csvfile)
        for row in reader:
            confusion[row[1]][row[2]] += 1

NameError: name 'encoder' is not defined

In [5]:
## Necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import warnings
warnings.filterwarnings("ignore")

# 1. TimeGAN model
from timegan import timegan
# 2. Data loading
from data_loading import real_data_loading, sine_data_generation
# 3. Metrics
from metrics.discriminative_metrics import discriminative_score_metrics
from metrics.predictive_metrics import predictive_score_metrics
from metrics.visualization_metrics import visualization

## Data loading
data_name = 'AF samples'
seq_len = 5000

print(data_name + ' dataset is ready.')

# Run TimeGAN
params = {
  'hidden_dim': 16,
  # hidden_dim changed from 24 to 16
  'num_layer': 3,
  'iterations': 10,
  'batch_size': 64,
  # batch_size changed from 128 to 64
  'module': 'gru'
}

generated_data = timegan(AF_files, params)
print('Finish Synthetic Data Generation')

# For Discriminative score
metric_iteration = 5

discriminative_score = list()
for _ in range(metric_iteration):
  temp_disc = discriminative_score_metrics(AF_files, generated_data)
  discriminative_score.append(temp_disc)

print('Discriminative score: ' + str(np.round(np.mean(discriminative_score), 4)))

# For Predictive score
predictive_score = list()
for tt in range(metric_iteration):
  temp_pred = predictive_score_metrics(AF_files, generated_data)
  predictive_score.append(temp_pred)   
    
print('Predictive score: ' + str(np.round(np.mean(predictive_score), 4)))

AF samples dataset is ready.
Start Embedding Network Training
step: 0/10, e_loss: 0.1285
step: 1/10, e_loss: 0.1081
step: 2/10, e_loss: 0.0892
step: 3/10, e_loss: 0.0698
step: 4/10, e_loss: 0.0517
step: 5/10, e_loss: 0.0374
step: 6/10, e_loss: 0.0237
step: 7/10, e_loss: 0.0234
step: 8/10, e_loss: 0.0299
step: 9/10, e_loss: 0.0362
Finish Embedding Network Training
Start Training with Supervised Loss Only
step: 0/10, s_loss: 0.0601
step: 1/10, s_loss: 0.0548
step: 2/10, s_loss: 0.0498
step: 3/10, s_loss: 0.0453
step: 4/10, s_loss: 0.0414
step: 5/10, s_loss: 0.0382
step: 6/10, s_loss: 0.0355
step: 7/10, s_loss: 0.0332
step: 8/10, s_loss: 0.0312
step: 9/10, s_loss: 0.0294
Finish Training with Supervised Loss Only
Start Joint Training
step: 0/10, d_loss: 2.0753, g_loss_u: 0.7051, g_loss_s: 0.0238, g_loss_v: 0.0394, e_loss_t0: 0.0261


In [2]:
import pickle

with open('AF_files.pkl', 'rb') as f:
    AF_records_orig = pickle.load(f)
    f.close()

with open('AF_files_resamp.pkl', 'rb') as f:
    AF_records_resamp = pickle.load(f)
    f.close()

print(AF_records_orig[0])
print(AF_records_resamp[0])

[[-18.067 ]
 [-25.015 ]
 [-30.766 ]
 ...
 [  2.3651]
 [  9.0081]
 [ 16.26  ]]
[[ -29.20850614]
 [-106.16174566]
 [-161.58552425]
 [ -69.94865016]
 [  19.95986007]
 [ -42.78278636]
 [  -1.1368875 ]
 [ -34.437712  ]
 [ -77.36429244]
 [  50.55591754]
 [-153.36930897]
 [   9.48252561]
 [  51.55097695]
 [-200.38604203]
 [-109.50966884]
 [-107.96249916]
 [  28.05872524]
 [ -12.41100331]
 [  57.37893206]
 [ -88.43549584]
 [-161.80720662]
 [-151.78428818]
 [   0.87399222]
 [ -41.09704905]
 [   1.13417921]
 [  32.08662868]
 [  31.01127469]
 [ -39.64454578]
 [  36.03585281]
 [-109.78096592]
 [-133.01903235]
 [-154.5762247 ]
 [ -90.88851681]
 [ -86.41351705]
 [   4.83716089]
 [   6.74338313]
 [  11.53772291]
 [  59.47320809]
 [ -78.68261402]
 [ -71.31105244]
 [-114.31876985]
 [  18.55798068]
 [ -38.24672744]
 [  12.98629604]
 [ -10.28320417]
 [ -63.28134298]
 [  26.94624729]
 [ -59.49181346]
 [-152.0869399 ]
 [-149.07278901]
 [-144.55824236]
 [-117.44135023]
 [ -77.91150069]
 [  36.4365858 ]
 [ -

In [19]:
len(AF_records_resamp[0])

500